In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<style>
body {
  background: #020617;
  color: white;
  font-family: Arial;
  text-align: center;
}

canvas {
  background: black;
  margin-top: 20px;
  border-radius: 10px;
}

.instrument {
  font-size: 16px;
  margin: 6px;
  padding: 6px;
  background: #0f172a;
  border-radius: 6px;
}

.instrument.active {
  background: #22d3ee;
  color: black;
}
</style>
</head>

<body>

<h2>🎵 Real-Time Instrument Frequency Detection</h2>

<input id="musicUrl" style="width:65%;padding:10px" placeholder="Paste MP3 link">
<button onclick="play()">Play</button>

<audio id="audio" crossorigin="anonymous"></audio>

<canvas id="canvas" width="900" height="300"></canvas>

<div id="detected"></div>

<script>
let ctx, analyser, dataArray, animation;

const instruments = [
  {name:"Kick / Sub-Bass", min:20, max:80},
  {name:"Bass Guitar", min:80, max:250},
  {name:"Drums / Percussion", min:50, max:300},
  {name:"Guitar", min:300, max:1200},
  {name:"Piano", min:100, max:4000},
  {name:"Violin", min:200, max:3500},
  {name:"Flute", min:500, max:2500},
  {name:"Brass", min:300, max:2000},
  {name:"Vocals", min:85, max:2550},
  {name:"Synth / Pads", min:100, max:8000}
];

function play() {
  const audio = document.getElementById("audio");
  audio.src = document.getElementById("musicUrl").value;
  audio.play();

  ctx = new AudioContext();
  analyser = ctx.createAnalyser();
  analyser.fftSize = 2048;

  const src = ctx.createMediaElementSource(audio);
  src.connect(analyser);
  analyser.connect(ctx.destination);

  dataArray = new Uint8Array(analyser.frequencyBinCount);
  draw();

  audio.onended = () => cancelAnimationFrame(animation);
}

function draw() {
  analyser.getByteFrequencyData(dataArray);

  const canvas = document.getElementById("canvas");
  const c = canvas.getContext("2d");
  c.clearRect(0,0,canvas.width,canvas.height);

  let detectedHTML = "";

  instruments.forEach(inst => {
    let energy = 0;
    let count = 0;

    for (let i=0;i<dataArray.length;i++) {
      let freq = i * ctx.sampleRate / analyser.fftSize;
      if (freq >= inst.min && freq <= inst.max) {
        energy += dataArray[i];
        count++;
      }
    }

    let avg = energy / (count || 1);
    let active = avg > 160;

    detectedHTML += `<div class="instrument ${active?'active':''}">
      ${inst.name} : ${inst.min}–${inst.max} Hz
    </div>`;
  });

  document.getElementById("detected").innerHTML = detectedHTML;

  animation = requestAnimationFrame(draw);
}
</script>

</body>
</html>
""")


In [ ]:
https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3

In [ ]:
import pickle
from google.colab import files

# Instrument frequency model (used by your project logic)
instrument_frequency_model = {
    "model_type": "Probable Instrument Detection (Frequency Based)",
    "instruments": [
        {"name": "Kick / Sub-Bass", "min_hz": 20, "max_hz": 80},
        {"name": "Bass Guitar", "min_hz": 80, "max_hz": 250},
        {"name": "Drums / Percussion", "min_hz": 50, "max_hz": 300},
        {"name": "Guitar", "min_hz": 300, "max_hz": 1200},
        {"name": "Piano", "min_hz": 100, "max_hz": 4000},
        {"name": "Violin", "min_hz": 200, "max_hz": 3500},
        {"name": "Flute", "min_hz": 500, "max_hz": 2500},
        {"name": "Brass Instruments", "min_hz": 300, "max_hz": 2000},
        {"name": "Vocals", "min_hz": 85, "max_hz": 2550},
        {"name": "Synth / Electronic", "min_hz": 100, "max_hz": 8000}
    ]
}

# Save as PKL file
file_name = "instrument_frequency_model.pkl"
with open(file_name, "wb") as f:
    pickle.dump(instrument_frequency_model, f)

# Download the PKL file
files.download(file_name)
